# Customer Churn - Telecom
---

### CRISP-DM Methodology
This project follows the CRISP-DM (*Cross-Industry Standard Process for Data Mining*) framework applied to **Customer Retention & Churn Prediction**:
| **Stage** | **Objective** | **Methodological Execution** |
| :--- | :--- | :--- |
| **1. Business Understanding** | Mitigate revenue loss by identifying at-risk customers. | • **Target Definition**: Binary Classification (Churn: Yes/No).<br>• **KPIs**: Maximize **Lift** in retention campaigns & Revenue Saved vs. Cost. |
| **2. Data Understanding** | Detect patterns of friction and dissatisfaction. | • **EDA**: Distribution analysis (Detect Imbalance).<br>• **Hypothesis Testing**: Correlation Matrix & Independence Tests (Chi-Square). |
| **3. Data Preparation** | Construct a robust dataset for parametric modeling. | • **Scaling**: Standardization (Z-score) for coefficient comparability.<br>• **Encoding**: One-Hot Encoding for nominal variables.<br>• **Splitting**: Stratified Train/Test Split to preserve class ratio. |
| **4. Modeling** | Estimate Churn Probability:<br>$$P(Y=1 \vert X) = \frac{1}{1+e^{-z}}$$ | • **Algorithm**: Logistic Regression (Baseline).<br>• **Inference**: Analyze **Odds Ratios** to determine feature elasticity. |
| **5. Evaluation** | Assess model reliability and financial impact. | • **Discrimination**: AUC-ROC & F1-Score (Threshold Tuning).<br>• **Calibration**: Probability Calibration Curve (Reliability Diagram). |
| **6. Deployment** | Integrate insights into the CRM lifecycle. | • **Deliverable**: "High-Risk" Customer List for Marketing Squad.<br>• **Artifact**: Serialize model (`joblib`) for batch inference. |

---
#### Note:

Although the CRISP-DM Modeling phase typically involves comparing several algorithms to select the best performer, this project focuses, by scope definition, on implementing a baseline. Therefore, a Logistic Regression model will be developed, going through all stages of the cycle (analysis, preparation, and modeling) to validate the initial hypothesis

### Installs:

In [0]:
%%capture
%pip install numpy==2.4.0
%pip install pandas==2.3.3
%pip install scikit-learn==1.8.0
%pip install matplotlib==3.10.8
%pip install seaborn==0.13.2

In [0]:
# Command to restart the kernel and update the installed libraries
%restart_python

### Imports:

In [0]:
# Data Analize and Visualization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Data Modeling / Model Linear / Metrics / Save Model
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, ConfusionMatrixDisplay
import joblib

### Dev objects

In [0]:
# ============================================================================= #
# >>> Module of functions and classes for creating graphs and visualizing data. #                                        
# ============================================================================= #

# ======================================================== #
# Imports:                                                 #
# ======================================================== #
# Graphics:
# Matplotlib
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns
# Data manipulation and visualization:
# Pandas
import pandas as pd
from scipy.stats import skew

# ======================================================== #
# Graphics Data - Class                                    #
# ======================================================== #

class GraphicsData:

    # Initialize Class
    def __init__(
        self, 
        data : pd.DataFrame,

    ):
        try:
            # Entry checks
            if data.empty:
                raise ValueError('The provided DataFrame is empty.')

            self.data = data

        except Exception  as e:
            print(f'[Error] Failed to load Dataframe : {str(e)}')

    # ======================================================== #
    # Initializer Subplot Grid - Function                      #
    # ======================================================== #
    def _initializer_subplot_grid(
        self, 
        num_columns, 
        figsize_per_row,
        h_size: int = 25
    ):
        """
        Initializes and returns a standardized matplotlib subplot grid layout.

        This utility method calculates the required number of rows based on 
        the number of variables in the dataset and the desired number of 
        columns per row. It then creates a grid of subplots accordingly and 
        applies a consistent styling.

        Args:
            num_columns (int): Number of subplots per row.
            figsize_per_row (int): Vertical size (height) per row in the final figure.

        Returns:
            tuple:
                - fig (matplotlib.figure.Figure): The full matplotlib figure object.
                - ax (np.ndarray of matplotlib.axes._subplots.AxesSubplot): Flattened array of subplot axes.
        """
        num_vars = len(self.data.columns)
        num_rows = (num_vars + num_columns - 1) // num_columns

        plt.rc('font', size = 12)
        fig, ax = plt.subplots(num_rows, num_columns, figsize = (h_size, num_rows * figsize_per_row))
        ax = ax.flatten()
        sns.set(style = 'whitegrid')

        return fig, ax
    
    # ======================================================== #
    # Finalize Subplot Layout - Function                       #
    # ======================================================== #
    def _finalize_subplot_layout(
        self,
        fig,
        ax,
        i: int,
        title: str = None,
        fontsize: int = 30,
    ):
        """
        Finalizes and displays a matplotlib figure by adjusting layout and removing unused subplots.

        This method is used after plotting multiple subplots to:
        - Remove any unused axes in the grid.
        - Set a central title for the entire figure.
        - Automatically adjust spacing and layout for better readability.
        - Display the resulting plot.

        Args:
            fig (matplotlib.figure.Figure): The matplotlib figure object containing the subplots.
            ax (np.ndarray of matplotlib.axes.Axes): Array of axes (flattened) for all subplots.
            i (int): Index of the last used subplot (all subplots after this will be removed).
            title (str, optional): Title to be displayed at the top of the entire figure.
            fontsize (int, optional): Font size of the overall title. Default is 30.
        """
        for j in range(i + 1, len(ax)):
                fig.delaxes(ax[j])
        
        plt.suptitle(title, fontsize = fontsize, fontweight = 'bold')
        plt.tight_layout(rect = [0, 0, 1, 0.97])
        plt.show()

    # ======================================================== #
    # Format Single AX - Function                              #
    # ======================================================== #
    def _format_single_ax(
        self,
        ax,
        title: str = None,
        fontsize: int = 20,
        linewidth: float = 0.9,
        feature_skew: float = 0,
        pct_outliers: float = 0
    ):

        """
        Applies standard formatting to a single subplot axis.

        This method configures a single axis by:
        - Setting the title with specified font size and bold style.
        - Hiding the x and y axis labels.
        - Adding dashed grid lines for both axes with configurable line width.

        Args:
            ax (matplotlib.axes.Axes): The axis to be formatted.
            title (str, optional): Title text for the axis. Defaults to None.
            fontsize (int, optional): Font size for the title. Defaults to 20.
            linewidth (float, optional): Width of the dashed grid lines. Defaults to 0.9.
        """
        ax.set_title(title, fontsize = fontsize, fontweight = 'bold')
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.grid(axis = 'y', which = 'major', linestyle = '--', linewidth = linewidth)
        ax.grid(axis = 'x', which = 'major', linestyle = '--', linewidth = linewidth)
        
        # Visual maker of normality (Skewness = 0)
        if abs(feature_skew) > 1:
            ax.set_facecolor('#fdf2e9') # Light orange background to indicate high asymmetry
        
        # Visual maker of Outliers (Pct > 5%)
        if pct_outliers > 5.0:
            ax.set_facecolor('#fffbe6')
      


    def numerical_histograms(
        self, 
        num_columns: int = 3,
        figsize_per_row: int = 6,
        color: str = '#3498db',
        hue: str = None,
        hue_order: list = None,
        palette: list = ['#1abc9c', '#ff6b6b'] ,
        title: str = 'Histograms of Numerical Variables',
    ):
        """
        Plots histograms with KDE (Kernel Density Estimation) for all numerical columns in the dataset.

        Optionally groups the histograms by a categorical target variable using different colors (hue).
        Useful for visualizing the distribution of numerical features and how they differ between groups.

        Args:
            num_columns (int): Number of plots per row in the subplot grid.
            figsize_per_row (int): Height of each row in inches (controls vertical spacing).
            color (str): Default color for histograms when `hue` is not specified.
            hue (str, optional): Name of the column used for grouping (e.g., 'churn_target'). Must be categorical.
            palette (list): List of colors for hue levels. Only used if `hue` is provided.
            title (str): Title of the entire figure layout.

        Raises:
            Exception: If plotting fails due to missing columns, incorrect types, or rendering errors.
        """
        try:

            # Order HUE
            if hue:
                hue_order = sorted(self.data[hue].dropna().unique())

            # Entry checks
            numeric_cols = self.data.select_dtypes(include = 'number').columns.tolist()
            if hue and hue in numeric_cols:
                numeric_cols.remove(hue)

            # Define AX and Fig
            fig, ax = self._initializer_subplot_grid(num_columns, figsize_per_row)

            
            for i, column in enumerate(numeric_cols):

                
                # Calculate Skewness for the title
                # dropna() is necessary for the statistical calculation not to fail
                feature_skew = self.data[column].dropna().skew()
                
                sns.histplot(
                    data = self.data,
                    x = column,
                    kde = True,
                    hue = hue,
                    hue_order = hue_order,
                    palette = palette if hue else None,
                    edgecolor = 'black',
                    line_kws = {'linewidth': 2},
                    alpha = 0.4 if hue else 0.7,
                    color = None if hue else color,
                    ax = ax[i],
                )

        
                # Line Means for Group
                if hue:

                    for j, target_val in enumerate(hue_order):

                        # Filter
                        subset_data = self.data[self.data[hue] == target_val][column]
                        mean_val = subset_data.mean()

                        # Color 
                        line_color = palette[j] if palette and j < len(palette) else 'black'

                        # Plot Line
                        ax[i].axvline(
                            mean_val,
                            color = line_color,
                            linestyle = '--',
                            linewidth = 2.5,
                            label = f'Mean({target_val}: {mean_val:.2f})'
                        )
                    ax[i].legend(fontsize = 10)

                # Config Ax's
                self._format_single_ax(ax[i], title = f'Variable: {column}. \n Skewness: {feature_skew:.2f}', feature_skew = feature_skew)
                

                    
            # Show Graphics
            self._finalize_subplot_layout(fig, ax, i, title = title)
        except Exception as e:
            print(f'[Error] Failed to generate numeric histograms: {str(e)}.')
    

    # ======================================================== #
    # Numerical Boxplots - Function                            #
    # ======================================================== #
    def numerical_boxplots(
        self, 
        hue: str = None, 
        num_columns: int = 3,
        figsize_per_row: int = 6,
        palette: list = ['#1abc9c', '#ff6b6b'],
        color: str = '#1abc9c',
        showfliers: bool = False,
        showmeans: bool = False,
        title: str = 'Boxplots of Numerical Variables',
        legend: list = []
    ):
        """
        Plots boxplots for each numerical variable in the dataset.

        Optionally groups the boxplots by a categorical hue variable (e.g., churn target), 
        allowing for comparison of distributions between groups. Helps identify outliers, 
        skewness, and variability in each feature.

        Args:
            hue (str, optional): Column name to group the boxplots (e.g., 'churn_target').
                                If None, individual boxplots are created without grouping.
            num_columns (int): Number of plots per row in the subplot grid.
            figsize_per_row (int): Height (in inches) of each row of plots.
            palette (list): Color palette to use when `hue` is provided.
            color (str): Single color to use when `hue` is not specified.
            showfliers (bool): Whether to display outlier points in the boxplots (default: False).
            title (str): Overall title for the subplot grid.
            legend (list): Custom legend labels to replace default tick labels when `hue` is present.

        Raises:
            ValueError: If the hue column is not found in the DataFrame.
            Exception: If plotting fails due to unexpected issues.
        """
        try:
            # Entry checks
            if hue and hue not in self.data.columns:
                raise ValueError(f"Column '{hue}' not in the DataFrame.")

            numeric_cols = self.data.select_dtypes(include = 'number').columns.tolist()
            if hue and hue in numeric_cols:
                numeric_cols.remove(hue)

            # Define AX and Fig
            fig, ax = self._initializer_subplot_grid(num_columns, figsize_per_row, h_size = 25)

            for i, column in enumerate(numeric_cols):

                    data_col = self.data[column].dropna()

                    # Calcule of Outliers (Rule of Tukey)
                    Q1 = data_col.quantile(0.25)
                    Q3 = data_col.quantile(0.75)
                    median = data_col.quantile(0.50)
                    IQR = Q3 - Q1
                    lower_fence = Q1 -1.5 * IQR
                    upper_fence = Q3 + 1.5 * IQR

                    # Count Outliers
                    outliers = data_col[(data_col < lower_fence) | (data_col > upper_fence)]
                    n_outliers = len(outliers)
                    pct_outliers = (n_outliers / len(data_col)) * 100

                    sns.boxplot(
                        data = self.data,
                        x = hue if hue else column,
                        y = column if hue else None,
                        hue = hue if hue else None,
                        palette = palette if hue else None,
                        color = None if hue else color,
                        showfliers = showfliers,
                        showmeans = showmeans,
                        #orient = 'h',
                        meanprops = {"marker": "o", "markerfacecolor": "white", "markeredgecolor": "black"},
                        fliersize = 3,
                        #legend = False,
                        ax = ax[i]
                    )

                    # Config Ax's
                    if len(legend) > 0:
                        ax[i].set_xticks([l for l in range(0, len(legend))])
                        ax[i].set_xticklabels(legend, fontsize = 14, fontweight = 'bold')
                    
                    if ax[i].get_legend():
                        ax[i].legend_.remove()
                    self._format_single_ax(ax[i], f'Variable: {column}\nOutliers: {n_outliers} ({pct_outliers:.1f}%)', pct_outliers = pct_outliers) 
                    ax[i].set_yticklabels([])
                    #sns.despine(ax = ax[i], top = True, right = True, left = True, bottom = True)
            
            # Show Graphics
            self._finalize_subplot_layout(fig, ax, i, title = title)
        except Exception as e: 
            print(f'[ERROR] Failed to generate numerical boxplots: {str(e)}.')
    
    # ======================================================== #
    # Categorical Countplots - Function                        #
    # ======================================================== #
    def categorical_countplots(
        self,
        hue: str = None,
        num_columns: int = 3,
        figsize_per_row: int = 7,
        palette: list = ['#1abc9c', '#ff6b6b'],
        color: str = '#8e44ad',
        title: str = 'Countplots of Categorical Variables '
    ):
        """
        Plots countplots for all categorical variables in the dataset.

        Optionally groups the bars using a hue column (e.g., 'churn_target'), allowing 
        visual comparison of class distributions between different categories. Annotates
        each bar with its percentage frequency.

        Args:
            hue (str, optional): Name of the column used to group bars (e.g., target variable).
                                If None, no grouping is applied.
            num_columns (int): Number of plots per row in the subplot grid.
            figsize_per_row (int): Height (in inches) of each subplot row.
            palette (list): List of colors to use when `hue` is specified.
            color (str): Default color to use when `hue` is not provided.
            title (str): General title for the entire plot grid.

        Raises:
            ValueError: If the hue column is not found in the DataFrame.
            Exception: If the plot generation fails for unexpected reasons.
        """
        try:
            # Entry checks
            if hue and hue not in self.data.columns:
                raise ValueError(f"Column '{hue}' not found in the DataFrame.")

            categorical_cols = self.data.select_dtypes(include = ['object', 'category', 'int8']).columns.tolist()
            if hue and hue in categorical_cols :
                categorical_cols.remove(hue)
            
            # Config Ax's
            fig, ax = self._initializer_subplot_grid(num_columns, figsize_per_row, h_size = 30)

            for i, column in enumerate(categorical_cols):
                sns.countplot(
                    data = self.data,
                    x = column,
                    hue = hue if hue else None,
                    palette = palette if hue else None,
                    color = None if hue else color,
                    edgecolor = 'white' if hue else 'black',
                    saturation = 1,
                    alpha = 0.8,
                    legend = False,
                    ax = ax[i]
                )
                
                total = len(self.data[column])
                for p in ax[i].patches:
                    height = p.get_height()
                    if height == 0:
                        continue
                    percentage = f'{100 * height / total:.1f}%'
                    x = p.get_x() + p.get_width() / 1.95
                    y = height
                    ax[i].annotate(
                        percentage,
                        (x, y),
                        ha = 'center',
                        va = 'bottom',
                        fontsize = 16,
                        fontweight = 'bold',
                        color = 'black'
                    )

                # Config Ax's
                self._format_single_ax(ax[i], f'Variable: {column}')
                ax[i].set_xticks(range(len(ax[i].get_xticklabels())))
                ax[i].set_xticklabels(ax[i].get_xticklabels(), fontsize = 16)
                
            # Show Graphics
            self._finalize_subplot_layout(fig, ax, i, title = title)
        except Exception as e:
            print(f'[ERROR] Failed to generate categorical countplots: {str(e)}')


    # ======================================================== #
    # Categorical Bar Percentages - Function                   #
    # ======================================================== #
    def categorical_bar_percentages(
        self,
        hue: str ,
        palette: list = ['#1abc9c', '#ff6b6b'],
        num_columns: int = 3,
        figsize_per_row: int = 8,
        title: str = 'Barplots Of The Individual Rate Percentages Of Each Column Class'
    ):
        """
        Plots barplots of churn percentages per class of each categorical variable.

        This method calculates the percentage distribution of a binary target (`hue`)
        within each category of all categorical columns in the dataset, and visualizes
        these percentages as barplots.

        Args:
            hue (str): Name of the binary target column (e.g., 'churn_target').
            palette (list, optional): List of colors for the hue classes.
                Defaults to ['#b0ff9d', '#db5856'].
            num_columns (int): Number of subplots per row in the grid.
            figsize_per_row (int): Height (in inches) allocated per subplot row.
            title (str): Overall title for the figure.

        Raises:
            ValueError: If `hue` is not found in the DataFrame.
            Exception: For other errors during computation or plotting.

        Returns:
            None: Displays the plot directly.
        """
        try:
            # Entry checks
            if hue and hue not in self.data.columns:
                raise ValueError(f"Column '{hue}' not found in the DataFrame.")
            categorical_cols = self.data.select_dtypes(include = ['object', 'category', 'int8']).columns.tolist()
            if hue and hue in categorical_cols:
                categorical_cols.remove(hue)

            # Define AX and Fig
            fig, ax = self._initializer_subplot_grid(num_columns, figsize_per_row, h_size = 30)

            for i, column in enumerate(categorical_cols):
                
                total_churn_per_class = self.data.groupby(column, observed = True)[hue].count().reset_index(name = f'total_count_class')

                result = (
                    self.data.groupby([column, hue], observed = True)[hue]
                    .count()
                    .reset_index(name = 'frequency')
                    .merge(total_churn_per_class, on = column)
                )
                result['percentage_per_class'] = round((result['frequency'] / result['total_count_class']) * 100, 2)

                sns.barplot(
                    data=result,
                    x = column,
                    y = 'percentage_per_class',
                    hue = hue,
                    palette = palette,
                    edgecolor = 'white',
                    saturation = 1,
                    legend = False,
                    ax = ax[i]
                )

                # Annotate bars
                for p in ax[i].patches:
                    height = p.get_height()
                    percentage = f'{height:.1f}%'
                    x = p.get_x() + p.get_width() / 2
                    ax[i].annotate(
                        percentage,
                        (x, height),
                        ha='center',
                        va='bottom',
                        fontsize=14,
                        fontweight = 'bold',
                        color='black'
                    )

                # Config Ax's
                self._format_single_ax(ax[i], f'Variable: {column}')
                ax[i].set_xticks(range(len(ax[i].get_xticklabels())))
                ax[i].set_xticklabels(ax[i].get_xticklabels(), fontsize = 16)
            
            # Show Graphics
            self._finalize_subplot_layout(fig, ax, i, title = title)
        except Exception as e:
            print(f'[ERROR] Failed to generate percentage barplots: {str(e)}.')

    # ======================================================== #
    # Target Analysis - Function                               #
    # ======================================================== #
    def plot_target_analysis(
        self, 
        target_col: str,
        title: str = 'Target Variable Distribution',
        colors: list =['#1abc9c', '#ff6b6b'],
        palette: str = 'RdYlBu_r',
        figsize: tuple = (14, 6)
    ):
        
        """
        Generates a panel for in-depth analysis of the target variable.
        """

        try:
            # Entry checks
            if target_col not in self.data.columns:
                raise ValueError(f"Target '{target_col}' not found in Dataframe.")

            # Prepraration of Data (Ordered by Frequency: Highest -> Lowest)
            counts = self.data[target_col].value_counts()
            labels = counts.index
            values = counts.values

            # Color Logic
            if len(labels) <=2:
                final_colors = colors
            else:
                final_colors = sns.color_palette(palette, n_colors = len(labels))

            # Define AX and Fig
            fig, ax = plt.subplots(1, 2, figsize = figsize)
            plt.suptitle(title, fontsize = 20, fontweight = 'bold', y = 1.02)

            # Plot 1: Donut Chart
            wedges, texts, autotexts = ax[0].pie(
                values, 
                labels = labels,
                autopct = '%1.1f%%',
                startangle = 90,
                colors = final_colors,
                pctdistance = 0.80,
                wedgeprops =  dict(width = 0.4, edgecolor = 'white'),
                textprops = {'fontsize': 14}
            )

            ax[0].set_title(f'Class Balance (%)', fontsize = 16, fontweight = 'bold')
            for autotext in autotexts:
                autotext.set_color('black')
                autotext.set_weight('bold')

            # Plot 2: Bar Plot
            sns.barplot(
                x = labels,
                y = values,
                hue = labels,
                palette = final_colors,
                order = labels,
                hue_order = labels,
                legend = False,
                ax = ax[1],
                edgecolor = 'black'
            )

            ax[1].set_title(f'Absolute Frequecy (N)', fontsize = 16, fontweight = 'bold')
            ax[1].set_ylabel('Count') 
            ax[1].grid(axis = 'y', linestyle = '--', alpha = 0.5)
            ax[1].grid(axis = 'x', linestyle = '--', alpha = 0.5)

            # Annotate of values for blarplot
            for p in ax[1].patches:
                height = p.get_height()
                if height > 0:
                    ax[1].annotate(
                    f'{int(height)}',
                    (p.get_x() + p.get_width() / 2., height),
                    ha = 'center', 
                    va = 'bottom',
                    fontsize = 14,
                    fontweight = 'bold',
                    color = 'black'
                    )
            
            plt.tight_layout(rect = [0, 0, 1, 0.97])
            plt.show()
        except Exception as e:
            print(f'[ERROR] Failed to plot target analysis: {str(e)}')

    # ======================================================== #
    # Correlation Heatmap - Function                           #
    # ======================================================== #
    def correlation_heatmap(
        self,
        title: str = None,
        cmap: str = 'RdBu_r',
        h_size: int = 20,
        v_size: int = 15
    ):
        """
        Plots a heatmap showing the correlation matrix among the numerical columns.

        This method computes the correlation matrix of the dataset and displays it as a heatmap,
        with annotations showing the correlation coefficients.

        Args:
            title (str, optional): Title for the heatmap plot. Defaults to None.
            cmap (str, optional): Colormap to use for the heatmap. Defaults to 'coolwarm'.

        Raises:
            Exception: If the heatmap generation or plotting fails.
        """
        try:
            # Select only the desired columns
            corr_data = self.data.corr()
            sns.set_style('white')
            # Define AX and Fig
            plt.rc('font', size = 15)
            fig, ax = plt.subplots(figsize = (h_size, v_size))
            
            sns.heatmap(
                corr_data,
                mask = np.triu(np.ones_like(corr_data, dtype = bool)),
                annot = True,
                cmap = cmap,
                center =  0,
                vmax = 1,
                vmin = -1,
                fmt = '.2f',
                linewidths = .5,
                cbar_kws = {'shrink': .8},
                ax = ax
            )
            # Config Ax's and Show Graphics
            ax.set_title(title, fontsize = 20, fontweight = 'bold')
            plt.tight_layout(rect = [0, 0, 1, 0.97])
            plt.show()
        except Exception as e:
            print(f'[Error] Failed to generate correlation heatmap: {str(e)}.')


### Load the data

In [0]:
df = pd.read_csv('../data/ChurnData.csv')

### Verify successful load with some randomly selected records


In [0]:
df.sample(9)

### 1. Business Understanding: 

#### 1.1 Situation Assessment
The Telecommunications industry faces saturation and fierce competition. With high Customer Acquisition Costs (CAC), the profitability strategy has shifted from **Acquisition** to **Retention**. 
Current analysis shows that losing a long-standing customer ("High Tenure") is 5x more expensive than acquiring a new one due to the loss of predictable recurring revenue (LTV).

#### 1.2 The Business Problem
The company is experiencing an unexplained increase in customer attrition (Churn). Traditional rule-based methods (e.g., "Cancel if usage drops 10%") are reactive and fail to capture complex behavioral patterns. 
The marketing team needs a proactive mechanism to identify at-risk customers **before** the cancellation decision is irreversible.

#### 1.3 Objectives
1.  **Primary Goal:** Mitigate revenue loss by accurately identifying customers with high probability of churning.
2.  **Strategic Goal:** Understand the *drivers* of churn (Explainability). Is it price (`wiremon`) or poor service? This informs the product roadmap.
3.  **Financial Goal:** Maximize the **ROI of Retention**.
    * *Constraint:* We cannot offer discounts to everyone (Cost of Intervention). We must target only high-risk/high-value customers.

#### 1.4 Success Criteria (KPIs)
Instead of purely technical metrics (Accuracy), success is defined by:
* **Lift in Top Decile:** The model must capture at least 30% of total churners within the top 10% risk list.
* **Precision (Cost of False Positives):** Minimizing "False Alarms" to avoid giving discounts to customers who would have stayed anyway.

### 2. Data Understanding:
---

#### Dataset: `Telecommunications Churn Database`

- This dataset captures a historical snapshot of customer demographics, service usage patterns, and account status. The primary objective is to correlate these features with the attrition event (**Churn**) to isolate behavioral drivers of dissatisfaction.
---

#### Variables Dictionary:

**1. Target Variable (The Outcome)**
* **churn** *Integer (Binary)* - The classification target. `1` = Customer left (Voluntary Churn); `0` = Stayed.

**2. Customer Demographics & Profile**
* **custcat** *Categorical* - Customer category classification (1-4). Represents the customer segment/cluster.
* **tenure** *Integer* - Months with the company (Proxy for *Loyalty*).
* **age** *Integer* - Customer's age in years.
* **address** *Integer* - Years living at current address (Stability indicator).
* **ed** *Categorical* - Education level (1-5).
* **employ** *Integer* - Years with current employer.
* **income** *Continuous* - Annual household income (in thousands).

**3. Service Subscriptions (Binary Portfolio)**
* **ebill** *Binary* - Electronic billing subscription (0/1). (Digital Adoption indicator).
* **equip** *Binary* - Equipment rental (0/1).
* **callcard** *Binary* - Calling card service (0/1).
* **wireless** *Binary* - Wireless service (0/1).
* **pager** *Binary* - Pager service (Legacy technology).
* **internet** *Binary* - Internet service (0/1).
* **voice** *Binary* - Voice mail service (0/1).
* **callwait** *Binary* - Call waiting service (0/1).
* **confer** *Binary* - Conference calling service (0/1).

**4. Billing & Usage (Monthly Dynamics)**
* **longmon** *Continuous* - Average monthly long-distance usage ($).
* **tollmon** *Continuous* - Average monthly toll-free usage ($).
* **equipmon** *Continuous* - Average monthly equipment rental charges ($).
* **cardmon** *Continuous* - Average monthly calling card charges ($).
* **wiremon** *Continuous* - Average monthly wireless service charges ($).



#### Exploratory Data Analysis (EDA):
---

#### Univariate Analysis:
---

Examines the behavior of **a single variable** in isolation to understand its distribution, central tendency, and dispersion (e.g., histograms and means), without seeking relationships with other data.

In [0]:
df.head()

In [0]:
df.info()

#### Adjusting the variable types with their respective characteristics. 
---
- In this data, there are both binary and ordinal variables; I will be adjusting them so that there is no invalid statistical aggregation in the analyses.

In [0]:
def optimize_dtypes(df):
    """
    Adjusting the variable types with their respective characteristics. In this data, there are both binary and ordinal variables; I will be adjusting them so that there is no invalid statistical aggregation in the analyses.
    """
    # Copy DF
    df_clean = df.copy()

    # Categorical Columns
    categorical_cols = ['custcat', 'ed',]

    # Interger Columns
    integer_cols =  ['tenure', 'age', 'address', 'employ']

    # Binary Columns
    binary_cols = [
        'equip','callcard', 'wireless','ebill', 'voice', 'pager',
        'internet', 'callwait', 'confer','churn'
    ]

    # Continus Columns
    continuos_cols = [
        'income',  'longmon', 'tollmon', 'equipmon', 'cardmon',
        'wiremon',
    ]

    # Categorical Columns
    df_clean[categorical_cols] = df_clean[categorical_cols].astype('int32')
    df_clean[categorical_cols] = df_clean[categorical_cols].astype('category')

    # Interger Columns
    df_clean[integer_cols] = df_clean[integer_cols].astype('Int64')

    # Binary Columns
    df_clean[binary_cols] = df_clean[binary_cols].astype('int8')

    # Continus Columns
    df_clean[continuos_cols] = df_clean[continuos_cols].astype('float32')

    return df_clean


df = optimize_dtypes(df)

print(f'New dtypes of variables:')
df.info()

print(f'Visual sample:')
df.head()

In [0]:
df.isnull().sum()

In [0]:
df[df.duplicated(keep = False)]

In [0]:
data_describe = df.describe()
data_describe

In [0]:
data_describe.loc['min']

In [0]:
data_describe.loc['max']

In [0]:
data_describe.loc['mean']

In [0]:
numericals_cols = [
    'tenure', 'age', 'employ', 'address', 'income', 'longmon', 'tollmon', 'equipmon', 'cardmon', 'wiremon',
]
GraphicsData(data = df[numericals_cols]).numerical_histograms()

In [0]:

GraphicsData(data = df[numericals_cols]).numerical_boxplots(showfliers = True, showmeans = True)

In [0]:
categorical_cols = [
    'ed', 'equip', 'callcard', 'wireless', 'voice', 'pager', 'internet',
    'callwait', 'confer', 'ebill', 'custcat'
]
GraphicsData(data = df[categorical_cols]).categorical_countplots()

In [0]:
GraphicsData(data = df).plot_target_analysis(target_col='churn', colors=['#1abc9c', '#ff6b6b'])

#### Key Observations:
---
**1. Data Integrity and Quality**
- The dataset exhibits high integrity, containing 200 records and 22 active variables. The absence of null or duplicate data eliminates the need for imputation, allowing immediate focus on statistical modeling.
---
**2. Distribution of Numerical Variables**

  - **Income:** There is extreme skewness (**9.96**). The scenario reflects a concentration of income: a vast majority with standard income and an elite of "Super Rich" (representing the **6.5% of outliers**). **Spending Pattern (`longmon`, `cardmon`, `wiremon`):** Consumption variables follow the income curve. The strong skewness confirms that spending behavior is directly proportional to purchasing power.

  - **Niche Services (`tollmon`, `equipmon`):** These distributions are "Zero-Inflated," indicating that a significant portion of the base does not even use these specific services.

  - **Demographics (`age`):** The base is balanced, with a predominance of customers up to 50 years old, but with a tail extending to 76 years old, suggesting demographic maturity.

  - **Loyalty (`tenure`):** The bimodal/flat distribution indicates a healthy life cycle: the company manages to attract new customers (entrants) while retaining a loyal older base.
---
**3. Outlier Analysis**
  - The volume of outliers is not critical to the size of the dataset. The discrepancies in `income` and `longmon` are not errors, but natural characteristics of high-value customers (Whales). *Strategy: Treat with mathematical transformations instead of removal.*
---
**4. Portfolio Analysis (Categorical Variables)**

  - **Educational Profile (`ed`):** Concentration in middle levels (2 and 4). Level 5 (Postgraduate) is a minority, deserving specific investigation regarding its retention.

  - **Low Adoption:** Products such as `wireless`, `voice`, and `pager` have penetration below **30%**. They are candidates for *Upsell* campaigns or product review.

  - **Flagship Product:** The `callcard` dominates with **70.05%** adoption, proving to be the company's essential entry-level product.

  - **Moderate Adoption:** `internet`, `ebill`, and `confer` are in the intermediate range. They are not niche products, but have great growth potential in the current base.
---
**5. Churn Rate (29%)** 
  - The cancellation rate of **29%** is at the upper limit of the telecommunications market average. Although the sector is volatile, losing almost 1/3 of the customer base annually requires immediate, data-driven retention actions.
---


#### Bi-Variate Analysis:
---

Explores the mathematical relationship between **two variables** simultaneously to discover associations, correlations, or dependencies (e.g., scatterplot of Income vs. Debt).

#### Note: 
---
- For the execution of the bivariate analysis, I will perform a prior division of the data into **Train** and **Test** sets. The primary objective is to avoid **Data Leakage**, ensuring that all insights, outlier treatments, and feature engineering are derived exclusively from the training set.

- In addition, I will use the `stratify` parameter in the `train_test_split` function. Since this is a classification problem (churn prediction), it is mandatory to maintain the same **proportion of classes** of the target variable in both subsets, preserving the original statistical representativeness.

In [0]:
train_set, test_set = train_test_split(df, test_size = 0.2, stratify = df['churn'], shuffle = True, random_state = 33)

In [0]:
# Checking the proportions of the target variable
print(f'Shape of training: {train_set.shape}')
print(f'Shape of test: {test_set.shape}')

print('\n--- Churn Rate (Stratify Validation) ---')
print(f'Original: {df['churn'].mean():.2%}')
print(f'Train:    {train_set['churn'].mean():.2%}')
print(f'Test:    {test_set['churn'].mean():.2%}')

In [0]:
train_set.head()

#### Checking the correlations between the variables

In [0]:
train_set.corr()['churn'].abs().sort_values(ascending = False)

In [0]:
GraphicsData(data = train_set).correlation_heatmap()


#### Key Observations:
---
**1. Retention Factors (Negative Correlation)**

   - **Tenure (`tenure`):** Shows the strongest negative correlation (**-0.35**) with Churn. This confirms the premise of "Survival Analysis": the probability of cancellation decreases drastically as customer tenure increases. The critical customer is the newcomer.

   - **Stability (`address`, `employ`, `age`):** Variables that denote life stability (time at address, stable employment, and advanced age) are strong protectors against Churn. The profile of a loyal customer is mature and conservative.

   - **Voice Usage (`longmon`, `callcard`):** Customers with high consumption of voice services (long distance and calling cards) tend to be more loyal. This suggests that the "Voice" product generates greater *lock-in* than digital products.
---
**2. Risk Factors (Positive Correlation)**

   - **Education (`ed`):** With a positive correlation (**0.24**), it is observed that customers with higher levels of education tend to cancel more. This suggests that more educated consumers are more demanding, research the competition more, and have a lower informational barrier to switching operators.

   - **Digital Services (`equip`, `internet`, `wireless`):** Paradoxically, the contracting of modern services is associated with **higher Churn**.

   - *Hypothesis:* These services operate in highly competitive markets (commodities), where the customer switches providers for small price differences, unlike traditional voice service.

   - **Digital Billing (`ebill`):** The positive correlation with Churn (**0.21**) suggests a behavioral profile. Users of `ebill` tend to be younger and more digitally savvy, possessing a lower switching cost and lower brand loyalty than users of physical invoices.
---
**3. Neutral and Categorical Variables**

- **Income:** The low linear correlation (-0.09) reflects the high asymmetry (Skewness 9.96). The relationship between money and churn is probably not linear, requiring transformations (Log) to reveal its true predictive power.

- **Category (`custcat`**): Being nominal, its influence will be validated via visual analysis, since Pearson does not capture nuances of non-ordinal categories.
---
**4. Multicollinearity Alert**
There are some critical redundancies between service ownership and monthly cost indicators:
   - **`equip` vs `equipmon`** (0.95)
   - **`wireless` vs `wiremon`** (0.89)
   - *For linear models (Logistic Regression), it will be mandatory to remove the continuous cost variables (`...mon`) in favor of the binary ones, or vice versa, to avoid instability in the coefficients.*

   - **`longmon` vs `ternure`** (0.77): This correlation means that older customers spend more on long-distance calling services. There is a possibility that the model will become unstable if these 2 variables are maintained, because the correlation between them, although not perfect, is significant.

   - **`address` vs `age`** (0.74): There is a hypothesis that they are passing similar information but not the same information. Older customers tend to stay longer at the same address, as this behavior relates to *Stability*.
   Contextually, it makes sense for one variable to have a linear relationship with the other. However, in this case, it is worth evaluating these two variables and their possible influence on the target variable `churn`.
---

#### Analyzing the numerical variables and their relationships with the target variable.

In [0]:
numericals_cols = [
    'tenure', 'age', 'employ', 'address', 'income', 'longmon', 'tollmon', 'equipmon', 'cardmon', 'wiremon', 'churn'
]
GraphicsData(data = df[numericals_cols]).numerical_histograms(hue = 'churn')

In [0]:
GraphicsData(data = df[numericals_cols]).numerical_boxplots(hue = 'churn', showmeans = True)

#### Key Observations:
---
**1. Customer Lifecycle (`tenure`)**

  - **Averages:** Churners (22.78 months) vs. Non-Churners (40.22 months).

  - **Survival Analysis:** There is a "Critical Risk Zone" between 0 and 22 months. The average churn rate at 22 months indicates that if the institution manages to retain the customer beyond this initial 2-year barrier, the probability of loyalty doubles.

  - **Retention Insight:** Customers who exceed the 50-month mark become practically immune to churn ("total lock-in"). The retention strategy should be aggressive in the first year (onboarding) to push the customer base into the safe zone of 40+ months.
---
**2. The Stability Factor (`age`, `employ`, `address`)**

  - **The Pattern:** There is a direct correlation between life stability and loyalty.

  - **Age:** Loyal customers are, on average, 8 years older (43 years) than churners (35 years). The age group over 65 years old shows almost zero churn.

  - **Employment:** The employment time of retained customers (12 years) is more than **double** that of those who cancel (5.5 years).

  - **Residence:** Address stability follows the same proportion (13 years vs. 7.5 years).

  - **Diagnosis:** Churn at this institution is not just a matter of dissatisfaction with the service, but a reflection of the **financial volatility** of the younger and more unstable demographic profile. Cancellation can often be involuntary (non-payment) or due to price sensitivity.
---
**3. The Paradox of Income and Digital Services (`income`, `wiremon`, `equipmon`)**

  - **Economic Scenario:** Customers who cancel have an average income 31% lower (56k) than loyal customers (82k).

  - **The Paradox:** Despite having lower income and less stability, churners are the ones who most often contract (and spend on) "premium" and modern services, such as wireless and equipment rental (`equipmon`).

  - **Business Conclusion:** There is a **Product-Market Fit** misalignment. Expensive products (Wireless/Equipment) are being sold to an audience with lower purchasing power and high instability (young people), generating default or rapid cancellation. Meanwhile, the loyal audience (wealthy and mature) consumes cheap and legacy products.
---
**4. Loyalty Anchor (`longmon`, `cardmon`)**

  - **Voice Products:** Unlike digital services, voice usage (long distance) is the major differentiator for loyalty. Retained customers spend almost twice as much (13.6 vs. 7.2) on long-distance calls.

  - **Insight:** The voice service acts as an "anchoring" product. Those who use the phone to talk, stay. Those who use it for internet/data, leave.
---
**5. Noise Variables (`tollmon`)**

- **Irrelevance:** The `tollmon` variable (toll/extra charges) does not present a clear statistical distinction between the means or distributions of the two groups. It is suggested to evaluate its removal (Feature Selection) to reduce the complexity of the model without loss of information.

---

#### Analyzing the categorical variables and their relationships with the target variable.

In [0]:
categorical_cols = [
    'ed', 'equip', 'callcard', 'wireless', 'voice', 'pager', 'internet',
    'callwait', 'confer', 'ebill', 'custcat', 'churn'
]
GraphicsData(data = df[categorical_cols]).categorical_countplots(hue = 'churn')

In [0]:
GraphicsData(data = df[categorical_cols]).categorical_bar_percentages(hue = 'churn')

#### Key Observations

---

**1. Educational Level (`ed`)**

* **The Effect of "Smart Switching":** A linear increase in the cancellation rate is observed as the educational level increases, reaching a critical peak of **47.1%** at Level 5.
* **Diagnosis:** Customers with a high level of education treat communications as a *commodity*. They have **low switching costs** and make rational decisions. Unlike the inert base, this profile compares market prices. The prevention strategy here should be based on financial advantage, not emotional.

---

**2. Digital Services and Equipment (`equip`, `internet`, `wireless`)**

* **The Toxic Trio:** Data reveals a structural flaw in these value-added services. Retention drops significantly when customers acquire them.

* **Internet:** The cancellation rate jumps from **18.8%** to **42.0%**.

* **Equipment:** Even more critical, churn explodes from **18.3%** to **43.5%**.

* **Product-Market Fit Alert:** This indicates that the company's digital infrastructure and hardware rental policies are acting as **churn accelerators**. Selling these products to at-risk segments (such as young users) is effective in pushing the competition.

---

**3. Voice Services (phone card, conferencing, voice)**

* **Retention Anchors (with an imposter):** Functional voice features generally generate high loyalty (retention), but there is a clear divide.

* **Phone card:** A massive retention factor. Users canceled significantly less (**19.9%**) than non-users (**50.9%**).

* **Conference and Call Waiting:** Both confirm the rule, showing lower churn rates for users.

* **The "Toxic" Intruder (`voice`):** Voicemail defies the category trend. Users canceled the service significantly more (**39.0%**) than non-users (**24.8%**). This suggests that a `voice` behaves more like a deficient "Digital Service" (generating friction/cost) than a useful "Voice Service".

---

**4. Customer Segmentation (`customer`)**

* **Scalability Alert:** The disparity between segments is extreme.

* **Class 3:** The "Gold Segment" with only **8.3%** cancellation rate.

* **Class 4:** The "Danger Zone" with a **45.6%** cancellation rate.

---

#### Multi-Variate Analysis:
---

Analyzes a set of **three or more variables** at the same time to understand complex interactions and latent structures.

#### Optimization of Numerical Variables

- Some financial and consumption variables have "long tails" (many clients with low values, few with extreme values), which distorts the predictive capacity of some algorithms.

- **Action:** A mathematical (Logarithmic) normalization will be applied to balance these distributions. Objective: To "unlock" hidden patterns of behavior, allowing the model to better differentiate at-risk clients, regardless of their income or consumption bracket.

In [0]:
# Columns for transfomations
skew_columns = ['income', 'longmon', 'cardmon', 'wiremon']

for col in skew_columns:
    train_set[f'log_{col}'] = np.log1p(train_set[col])

In [0]:
train_set.head()

In [0]:
skew_columns = ['income', 'longmon', 'cardmon', 'wiremon', 'log_income', 'log_longmon', 'log_cardmon', 'log_wiremon', 'churn']
GraphicsData(data = train_set[skew_columns]).numerical_histograms(hue = 'churn')

In [0]:
skew_columns = ['income', 'longmon', 'cardmon', 'wiremon', 'log_income', 'log_longmon', 'log_cardmon', 'log_wiremon', 'churn']
GraphicsData(data = train_set[skew_columns]).correlation_heatmap()


#### Note:
---
**Impact Analysis: Log-Transformation**

- **Mitigation of Skewness:** The application of the `log1p` transformation was effective in normalizing the distributions, drastically reducing the skewness of the critical variables.

- **Sign Gain (Pearson Correlation):** There was a tangible increase in linearity with the target variable (`churn`):
- **Income:** Increase from **0.09** to **0.13** (+44% relative gain).

- **Longmon:** Refinement from **0.26** to **0.29**.

- **Cardmon (Highlight):** The most significant jump, doubling its relevance from **0.13** to **0.24**.

- **Discovery of Latent Patterns:** The transformation in `cardmon` revealed a hidden **bimodal** structure in the raw data. The logarithm visually separated two distinct subgroups of behavior:

  - 1. **Peak on the Left:** Casual Users (*Low usage*).

  - 2. **Peak on the Right:** Heavy Users (*Heavy users*).

*This will facilitate the creation of decision cuts by tree-based models.*

---

#### Creating News Features

In [0]:
# Aggregations of costs (Total Wallet Share)
# All expenses related to monthly services will be added together.
mon_cols = ['longmon', 'tollmon', 'equipmon', 'cardmon','wiremon']
train_set['total_spend'] = train_set[mon_cols].sum(axis = 1)

# Accessibility Index (Affordability)
# Since 'income' is in thousands (e.g., 20 = 20,000), I adjusted the scale.
# I added +1 to the denominator to avoid division by zero (safety).
train_set['affordability_idx'] = train_set['total_spend'] / ((train_set['income'] * 1000) + 1)

# Risk Feature (Toxicity + Education)
toxic_list = ['internet', 'wireless', 'equip', 'voice', 'pager']
train_set['toxic_score'] = train_set[toxic_list].sum(axis = 1)
train_set['toxic_ed'] = (train_set['toxic_score'] * train_set['ed'].astype('int64')).astype('float32')

# Behavioral Usage Features
# Ternure for longom
train_set['ternure_longmon'] = ((train_set['tenure'] / 12) * (train_set['longmon']) ).astype('float32')
# Ternure for cardmon
train_set['ternure_cardmon'] = ((train_set['tenure'] / 12) * (train_set['cardmon']) ).astype('float32')

# Stability Features
# Ternure for age 
train_set['stability_age'] =  ((train_set['tenure'] / 12) * (train_set['age'] - 18) ).astype('float32')
# Ternure for address 
train_set['stability_address'] = ((train_set['tenure'] / 12) * (train_set['address']) ).astype('float32')
# Ternure for address 
train_set['stability_employ'] = ((train_set['tenure'] / 12) * (train_set['employ']) ).astype('float32')

# Stability Feature (The Master Feature)
factors = train_set['address'] + train_set['age'] + train_set['employ']
train_set['stability_full'] = (factors * (train_set['tenure'] / 12)).astype('float32')

# Good Score 
train_set['good_score'] = train_set[['callcard', 'confer', 'callwait']].sum(axis=1)

In [0]:
new_features =  [ 'log_income', 'log_longmon', 'log_cardmon',
       'log_wiremon', 'total_spend', 'affordability_idx', 'toxic_score',
       'toxic_ed', 'ternure_longmon', 'ternure_cardmon', 'stability_age',
       'stability_address', 'stability_employ', 'stability_full',
       'good_score', 'churn',]
GraphicsData(data = train_set[new_features]).numerical_histograms(hue = 'churn')

#### Features Selections

#### Numerical Features
---

#### Mann-Whitney U Test
---
- For the Feature Selection stage, I adopted the Mann-Whitney U Test. This choice is due to the high skewness (> 1) of the numerical variables, which requires a non-parametric approach robust to outliers, where traditional tests (such as the T-test) would fail.
---

In [0]:
# ============================================================================= #
# >>> Module of functions and classes for creating testing data. #                                        
# ============================================================================= #

# ======================================================== #
# Imports:                                                 #
# ======================================================== #
# Data manipulation and testing:
# Pandas
import pandas as pd
# Scipy
from scipy.stats import mannwhitneyu, pointbiserialr, chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
# Numpy
import numpy as np

class EDATest:

# Initialize Class
    def __init__(
        self, 
        data: pd.DataFrame,
    ):
        try:
            # Entry checks
            if data.empty:
                raise ValueError('The provided DataFrame is empty.')

            if data.isnull().any().any():
                raise ValueError('The DataFrame contains null data.')

            self.data = data

        except Exception  as e:
                print(f'[Error] Failed to load Dataframe : {str(e)}')


    def mannwhitney_u_test(
        self,
        audit_vars: list,
        target: str,
    ):
        
        try:

            results_audit = []

            for var in audit_vars:

                if var not in self.data.columns:
                    continue
                
                # Separating the groups
                group_churn = self.data[self.data[target] == 1][var]
                group_no_churn = self.data[self.data[target] == 0][var]
                
                # Mann-Whitney U Test
                try:
                    stat, p_val = mannwhitneyu(group_churn, group_no_churn, alternative = 'two-sided')
                
                except:
                    p_val = 1.0
                
                # Linear Correlation (Point Biserial)
                try:
                    corr, _ = pointbiserialr(self.data[target], self.data[var])
                
                except:
                    corr = 0

                # Effect Size (Median Lift)
                med_churn = group_churn.median()
                med_loyal = group_no_churn.median()
                # Calculete of lift_median
                if med_loyal == 0:
                    lift_median = med_churn
                else:
                    lift_median = ((med_churn - med_loyal) / med_loyal) * 100
                
                # Diagnosis/ Classification
                if p_val < 0.001:
                    strength = '⭐⭐⭐ Strong'

                elif p_val < 0.05:
                    strength =  '⭐⭐ Medium'
                else:
                    strength = '❌ Noise'
                
                # Results
                results_audit.append({
                    'Feature': var,
                    'Strength': strength,
                    'P-Value': round(p_val, 5),
                    'Pearson Corr': round(corr, 3),
                    'Median Churn': round(med_churn, 2),
                    'Median Loyal': round(med_loyal, 2),
                    'Impact (%)': round(lift_median, 1), 
                })
            
            # Sort by Absolute Correlation (to see who has the greatest impact, positive or negative)
            df_audit = pd.DataFrame(results_audit)
            df_audit['Abs_Corr'] = df_audit['Pearson Corr'].abs()
            df_audit = df_audit.sort_values(by = 'Abs_Corr', ascending = False).drop(columns = ['Abs_Corr'])
        
            # Display
            print('Table of Scientific Evidence:')
            display(df_audit)

        except Exception as e:
            print(f'[Error] Failure to generate statistical tests: {str(e)}')
    

    def vif_test(
        self,
        audit_vars: list,
    ):
        
        try:
            
            # Selection features for test
            data = self.data[audit_vars].copy()

            for col in data.columns:
                # Ensures that booleans (True/False) reach 1/0
                if data[col].dtype == 'bool':
                    data[col] =  data[col].astype(int)
                
                # Force the numerical transformation to avoid errors in the test
                data[col] = pd.to_numeric(data[col], errors = 'coerce')
            # Cleaning data
            data.replace([np.inf, -np.inf], np.nan, inplace = True)
            data.dropna(inplace = True)
            print(f'Rows remaining after cleaning: {len(data)}')

            # Calculating the VIF
            data = data.astype(float)
            x_vif = add_constant(data)

            vif_data = pd.DataFrame()
            vif_data['Feature'] = x_vif.columns
            vif_data['VIF'] = [variance_inflation_factor(x_vif.values, i) for i in range(len(x_vif.columns))]   

            print('\n----- VIF Result -----') 
            print(vif_data.sort_values('VIF', ascending = False))
        
        except Exception as e:
            print(f'[Error] Failure to generate VIF test: {str(e)}')
    
    def _cramers_v(
        self,
        x,
        y
    ):
        try:

            confusion_matrix = pd.crosstab(x, y)
            if confusion_matrix.shape[0] < 2 or confusion_matrix.shape[1] < 2:
                return 0 
            chi2 = chi2_contingency(confusion_matrix)[0]
            n = confusion_matrix.sum().sum()
            phi2 = chi2 / n
            r, k = confusion_matrix.shape
            with np.errstate(divide = 'ignore', invalid = 'ignore'):
                phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
                rcorr = r - ((r - 1) **2) / (n - 1)
                kcorr = k - ((k - 1) **2) / (n - 1)
                if min((kcorr - 1), (rcorr - 1)) == 0:
                    return 0
                return np.sqrt(phi2corr / min((kcorr -1), (rcorr -1)))
            
        except Exception as e:
            print(f'[Error] Failed to execute the function _cramers_v: {str(e)}')
    
    def chi_square_test(
        self, 
        audit_vars: list,
        target: str
    ):
        try:

            # List of result
            results_audit = []

            for var in audit_vars:
                # Check for existence
                if var not in self.data.columns:
                    continue
                
                # Contingency Table (Crossover)
                crosstab = pd.crosstab(self.data[var], self.data[target])

                # Statistical Test (Chi-Square)
                chi2, p_val, dof, expected = chi2_contingency(crosstab)

                # Strength of Association (Cramér's V replaces Pearson)
                assoc = self._cramers_v(self.data[var], self.data[target])
                
                # Calculates the churn rate for each category
                # Axis 1 sums (Churn 0 + Churn 1) to get the category total
                churn_rates = crosstab[1] / crosstab.sum(axis = 1)

                # Indetify the extremes
                # Category with the most churn
                risky_cat = churn_rates.idxmax()                
                # Rate %
                risky_rate = churn_rates.max() * 100
                
                # Category with the least churn
                safe_cat = churn_rates.idxmin()                
                # Rate %
                safe_rate = churn_rates.min() * 100

                # Impact: The difference in risk between the worst and best-case scenarios
                risk_gap = risky_rate - safe_rate
                
                # Diagnosis/ Classification
                if expected.min() < 5:
                    strength = '⚠️ Low sample'
                elif p_val < 0.001:
                    strength = '⭐⭐⭐ Strong'
                elif p_val < 0.05:
                    strength = '⭐⭐ Median'
                else:
                    strength = '❌ Noise'

                # Results
                results_audit.append({
                    'Feature': var,
                    'Strength': strength,
                    'P-Value': round(p_val, 5),
                    'Assoc. (Cramer V)': round(assoc, 3),
                    'Worst Category': f'{risky_cat} ({risky_rate:.0f}%)',
                    'Best Category': f'{safe_cat} ({safe_rate:.0f}%)',
                    'Impact (Gap %)': round(risk_gap, 1)
                })

            # Ordering by Absolute Association (Cramér's V)
            df_audit_cat = pd.DataFrame(results_audit)
            df_audit_cat = df_audit_cat.sort_values(by = 'Assoc. (Cramer V)', ascending = False)

            # Display
            print('Table of Scientific Evidence (Categorical):')
            display(df_audit_cat)

        except Exception as e:
            print(f'[Error] Failure to generate statistical tests: {str(e)}')   

    
    def _correlation_ratio(
        self,
        categories, 
        measurements
    ):
        
        try:

            # Forcing and ensuring correct typing
            cat = pd.Series(categories).astype(str)
            meas = pd.to_numeric(measurements, errors = 'coerce')

            # Security Cleanup (Removes NaNs generated during conversion)
            # Creates a mask where both data sets are valid
            valid_mask = (~cat.isna()) & (~meas.isna())
            cat = cat[valid_mask]
            meas = meas[valid_mask]

            if len(meas) == 0: return 0.0

            # Calcule of means
            y_total_avg = meas.mean()
            y_avg_per_cat = meas.groupby(cat, observed = True).mean()

            # Maps the category average back to each observation.
            y_avg_array = cat.map(y_avg_per_cat)

            # Calcule of Variance (Eta)
            numerator = np.sum((y_avg_array - y_total_avg) ** 2)
            denominator = np.sum((meas - y_total_avg) ** 2)

            if denominator == 0: return 0.0
            return np.sqrt(numerator / denominator)
        
        except Exception as e:
            print(f'[Error] Failed to execute the _correlation_ratio: {str(e)}')

    def mixed_redundancy_test(
        self,
        audit_pairs: list,
    ):
        
        try: 

            audit_results = []

            for cat_col, num_col in audit_pairs:
                if cat_col in self.data.columns and num_col in self.data.columns:

                    # Executes the mixed redundancy test
                    eta_score = self._correlation_ratio(self.data[cat_col], self.data[num_col])

                    # Rules of decison
                    decision = '✅ Keep'
                    action = '-'

                    if eta_score > 0.95:
                        decision = '🔴 CRITIC (Redundant)'
                        action = f'Dropt to `{cat_col}` or `{num_col}`'
                    
                    elif eta_score > 0.80:
                        decision = '⚠️ Alert (Strong)'
                        action = f'Evaluate removal'

                    # Results
                    audit_results.append({
                        'Categorical (X)': cat_col,
                        'Numerical (Y)': num_col,
                        'Eta Score':  round(eta_score, 4),
                        'Diagnosis' : decision,
                        'Action': action
                    })
            
            # Display
            df_audit = pd.DataFrame(audit_results).sort_values(by = 'Eta Score', ascending = False)
            display(df_audit)

        except Exception as e:
            print(f'[Error] Failure to generate statistical tests: {str(e)}')  


In [0]:
audit_vars = [
    'tenure', 'age', 'address', 'income', 'employ', 
    'longmon', 'tollmon', 'equipmon', 'cardmon',
    'wiremon','log_income', 'log_longmon', 'log_cardmon',
    'log_wiremon', 'total_spend', 'affordability_idx', 'toxic_score',
    'toxic_ed', 'ternure_longmon', 'ternure_cardmon', 'stability_age',
    'stability_address', 'stability_employ', 'stability_full',
    'good_score'  
]
EDATest(data = train_set).mannwhitney_u_test(audit_vars =  audit_vars, target = 'churn')

#### Conclusion of Test
---
- In this selection stage, variables with a P-value greater than 0.05 in the Mann-Whitney U test were excluded, indicating a lack of statistical significance. The case of the variable `income` is illustrative: even after the logarithmic transformation (`log_income`), the variable failed to reject the null hypothesis, proving that purchasing power is not a relevant discriminator of churn for this base.

- **In contrast**, the analysis robustly validated that retention is governed by **stability factors** (tenure, age, address) and by **service quality** (toxicity), which demonstrated a direct and statistically significant influence on customer retention.

In [0]:
audit_vars = [
    'tenure', 'age', 'address', 'employ', 
    'longmon', 'equipmon', 'cardmon',
    'wiremon', 'log_longmon', 'log_cardmon',
    'log_wiremon', 'toxic_score',
    'toxic_ed', 'ternure_longmon', 'ternure_cardmon', 'stability_age',
    'stability_address', 'stability_employ', 'stability_full', 'churn'
]
GraphicsData(train_set[audit_vars]).correlation_heatmap(v_size = 15, h_size = 20)

#### Note:
---
- The creation of interaction variables generated mathematical redundancy in the dataset (multicollinearity). To mitigate this, I adopted the **'Kill the Parent'** strategy: remove the original variables when their derived versions ('children') show greater suitability to the Target.

- As the new features better capture customer behavior, the following original variations were altered and removed to avoid noise in the model:

- *Cut-off list:* `tenure`, `age`, `address`, `employ`, `longmon`, `cardmon`, `log_longmon`, `log_cardmon`, `logwiremon`, `toxic_score`.
---

#### Multicollinearity Diagnosis (VIF Analysis)
---
- During the *Feature Engineering* phase, multiplicative interaction variables were created (e.g., `tenure` * `longmon`) to capture the elasticity of customer behavior over time. However, the introduction of these derived variables generates, by definition, a high degree of linear correlation with their original variables (Linear Dependence).

- To mitigate the risk of redundancy and ensure the parsimony of the model (Occam's Razor), I will apply the **VIF (Variance Inflation Factor)** test. The goal is to identify and remove variables with a VIF > 10, ensuring that the final model prioritizes the real *Feature Importance* and does not suffer from instability in the estimation of the coefficients.
---

In [0]:
audit_vars = [
    'equipmon', 'wiremon', 'toxic_ed', 'ternure_longmon', 'ternure_cardmon', 'stability_age',
    'stability_address', 'stability_employ', 'stability_full',
]

In [0]:
EDATest(train_set).vif_test(audit_vars = audit_vars)

In [0]:
audit_vars = [
    'equipmon', 'wiremon', 'toxic_ed', 'ternure_longmon', 'ternure_cardmon', 'stability_age',
    'stability_address', 'stability_employ', #'stability_full',
]

In [0]:
EDATest(train_set).vif_test(audit_vars = audit_vars)

#### Note:
---
- **Variance Inflation Factor (VIF)** analysis detected severe multicollinearity in the derived feature `stability_full` (**VIF: 149.18**), indicating mathematical redundancy with its component variables (`stability_age`, `address`, `employ`).

  **Action:** Removal of the variable `stability_full`.

  **Result:** After deletion, the system rebalanced, with all remaining features showing **VIF < 10**.

- Although `stability_full` showed a high linear correlation with the target, I chose to keep the component variables (such as `stability_employ`). These demonstrated a superior **Impact (Class Separation)** and offer greater granularity for decision-making. For Decision Tree/Ensemble algorithms, the ability for pure segregation (Information Gain) surpasses the aggregate linear correlation.
---

#### Categorical Features

#### Chi Square Test

In [0]:
audit_vars = [
    'ed', 'equip', 'callcard', 'wireless', 'voice', 'pager', 'internet',
    'callwait', 'confer', 'ebill', 'custcat'
]
EDATest(train_set).chi_square_test(audit_vars = audit_vars, target = 'churn')

In [0]:
train_set['ed'].value_counts()

In [0]:
train_set['ed'] = train_set['ed'].astype('int64').replace({5: 4})
train_set['ed'] = train_set['ed'].astype('category')

In [0]:
EDATest(train_set).chi_square_test(audit_vars = audit_vars, target = 'churn')

#### Checking mixed correlations of categorical features and numerical features

In [0]:
audit_pairs = [
    ('equip', 'equipmon'),       
    ('wireless', 'wiremon'),     
    ('callcard', 'ternure_cardmon'),     
    ('ed', 'toxic_ed'),         
    ('internet', 'toxic_ed'),
    ('ebill', 'stability_age')
]
EDATest(train_set).mixed_redundancy_test(audit_pairs = audit_pairs)

### 3. Data Preparation:

#### Selecting variables for training and test data

In [0]:
X_train = train_set[['ENGINESIZE','FUELCONSUMPTION_COMB']]
y_train = train_set['CO2EMISSIONS']

print(f'The shape of X_train is: {X_train.shape}')
print(f'\nThe shape of y_train is: {y_train.shape}')

In [0]:
X_test = test_set[['ENGINESIZE','FUELCONSUMPTION_COMB']]
y_test = test_set['CO2EMISSIONS']

print(f'The shape of X_train is: {X_test.shape}')
print(f'\nThe shape of y_train is: {y_test.shape}')

#### Preprocessing

#### Note:
---
- For the preprocessing stage, opt for the application of the `PowerTransformer` (Yeo-Johnson method). This application, a parametric power transformation technique, evolves to **stabilize the variance** and approximate the distribution of predictors to a Normal (Gaussian) distribution. The method acts by mitigating the positive skewness (**long tail**) and correcting the **heteroscedasticity** identified in the `ENGINESIZE` variable, thus ensuring the fulfillment of the statistical predictions of the linear model.

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# --- 1. Definição das Variáveis (Feature Selection) ---

# A. Variáveis para EXCLUIR (Redundantes/Derivadas)
drop_cols = [
    'longten', 'tollten', 'cardten', # Totais (Interação Preço x Tempo)
    'lninc', 'loglong', 'logtoll', 'logturn', # Logs (Matemáticas)
    'callwait', 'confer', 'ebill', # Opcionais: Se quiser simplificar o modelo (Mantenha se achar relevante)
    'churn' # O Target jamais entra no X
]

# B. Variáveis Numéricas (Precisam de Z-Score)
numeric_features = [
    'tenure', 'age', 'address', 'income', 'employ', 
    'longmon', 'tollmon', 'equipmon', 'cardmon', 'wiremon'
]

# C. Variáveis Binárias (Já estão prontas: 0/1)
binary_features = [
    'equip', 'callcard', 'wireless', 'pager', 'internet', 'voice'
]

# D. Variáveis Categóricas/Ordinais (Precisam de Dummies)
# 'custcat' e 'ed' são números que representam categorias
categorical_features = ['ed', 'custcat'] 

# --- 2. Separação X e y ---

# Garante que estamos usando apenas as colunas que sobraram após o filtro mental
selected_features = numeric_features + binary_features + categorical_features

X = df[selected_features]
y = df['churn'] # O Target isolado

# Split Estratificado (Mantém a proporção de Churn no Treino e Teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# --- 3. Construção do Pipeline Robusto ---

preprocessor = ColumnTransformer(
    transformers=[
        # Numéricas: Padronização
        ('num', StandardScaler(), numeric_features),
        
        # Categóricas: One-Hot (drop='first' remove a colinearidade)
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features),
        
        # Binárias: Passar direto
        ('bin', 'passthrough', binary_features)
    ],
    verbose_feature_names_out=False # Mantém nomes limpos (ex: 'ed_2' em vez de 'cat__ed_2')
)

# Pipeline Final
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        solver='liblinear', # Ótimo para datasets pequenos/médios
        penalty='l1',       # Lasso: Ajuda a zerar coeficientes inúteis (Feature Selection automático)
        C=1.0,              # Inverso da força de regularização
        random_state=42
    ))
])

# --- 4. Treinamento ---
model_pipeline.fit(X_train, y_train)

# Feedback Visual
print("Pipeline treinado com sucesso.")
print(f"Total de Features de Entrada: {X.shape[1]}")
print(f"Total de Coeficientes Gerados (após One-Hot): {len(model_pipeline.named_steps['classifier'].coef_[0])}")

In [0]:
X_scaler = PowerTransformer(method = 'yeo-johnson')
y_scaler = PowerTransformer(method = 'yeo-johnson')

# Train data
X_train_preprocessed  = X_scaler.fit_transform(X_train)

# Test data 
X_test_preprocessed = X_scaler.transform(X_test)

# Label data
y_train_preprocessed  = y_scaler.fit_transform(y_train.values.reshape(-1, 1))

# Test data 
y_test_preprocessed = y_scaler.transform(y_test.values.reshape(-1, 1))

#### Train Data Preprocessed

In [0]:
pd.DataFrame(X_train_preprocessed, columns = X_scaler.get_feature_names_out(X_train.columns)).head()

#### Test Data Preprocessed

In [0]:
pd.DataFrame(X_test_preprocessed, columns = X_scaler.get_feature_names_out(X_test.columns)).head()

### 4. Modeling:

#### Cross-Validation

In [0]:
# Create model and  K-Fold
model = LinearRegression()
kfold = KFold(n_splits = 5, shuffle = True, random_state = 33)

# Create Cross-Validation
cv_results = cross_validate(
    model, 
    X_train_preprocessed,
    y_train_preprocessed,
    cv = kfold,
    scoring = 'r2',
    return_estimator = True
)

# Extraction of coefficients
coefs_list = []
for estimator in cv_results['estimator']:
    coefs_list.append(estimator.coef_.flatten())

# Converts to a NumPy array for easier statistical analysis (Shape: [5, n_features])
coefs_array = np.array(coefs_list)

# Stability Calculation (Audit)
coefs_mean = np.mean(coefs_array, axis = 0)
coefs_std = np.std(coefs_array, axis = 0)

# Metrics
print(f'--- Performance Metrics ---')
print(f'Mean R² {np.mean(cv_results['test_score']):.4f}')
print(f'Std R²: {np.std(cv_results['test_score']):.4f}')

print(f'\n--- Stability Metrics (Coefficients) ---')
feature_names = ['ENGINESIZE', 'FUELCONSUMPTION_COMB']
df_stability = pd.DataFrame(
    {
        'Feature': feature_names,
        'Mean Coef': coefs_mean,
        'Std Coef': coefs_std,
        'CV (%)': (coefs_std / np.abs(coefs_mean)) * 100
    }
)

print(df_stability)

#### Final Training

In [0]:
model.fit(X_train_preprocessed, y_train_preprocessed)

print(f'Coefficients: {model.coef_[0]}')
print(f'Intercept: {model.intercept_}')

#### Test Model

In [0]:
y_pred = model.predict(X_test_preprocessed)

#### Metrics:

In [0]:
# 1. Bring the test Y and the predicted Y back to the "Real World"
# The inverse_transform requires a 2D array, hence the reshape
y_pred_real = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_real = y_scaler.inverse_transform(y_test_preprocessed)

#2. Calculate metrics on the REAL scale (Grams of CO2)
mae_real = mean_absolute_error(y_test_real, y_pred_real)
rmse_real = root_mean_squared_error(y_test_real, y_pred_real)
r2_real = r2_score(y_test_real, y_pred_real)

print(f'--- Business Metrics (Original Scale) ---')
print(f"MAE Real: {mae_real:.2f} g/km")
print(f"RMSE Real: {rmse_real:.2f} g/km")
print(f"R2 Real: {r2_real:.4f}")

print(f"\n--- Statistical Metrics (Yeo-Johnson Scale) ---")
print(f"R2 Transformed: {r2_score(y_test_preprocessed, y_pred):.4f}")

#### Key Observations:
---

- 1. **Cross-Validation:** The application of Cross-Validation demonstrated exceptional stability in the model. The standard deviation of only **0.0170** between the k-folds confirms that the performance (average R² of **0.899**) is consistent and robust, minimizing the risk of sampling bias.
---

- 2. **Generalization Test:** In the test data, the model achieved a **Transformed R² of 0.91** (and **0.88** in the Real Scale). This transformed score, slightly higher than the training score (**0.90**), confirms that there was absolutely no *Overfitting*. The model learned the underlying physics of the data rather than memorizing noise.
---
- 3. **Stability (CV%):** **(`ENGINESIZE`)**: **4.05%** and **(`FUELCONSUMPTION_COMB`)**: **1.92%**. The CV is drastically below the 20% threshold, indicating "State-of-the-Art" stability. The **multicollinearity** (0.82 correlation) was effectively neutralized. The model assigned a clear, unwavering weight to the Fuel Consumption (Mean Coef ~0.71) as the dominant factor, while maintaining Engine Size (Mean Coef ~0.27) as a stable secondary predictor.
---
#### Insight:
---
---
- The convergence between the training **R² (0.90)** and the transformed test **R² (0.91)** validates the **Yeo-Johnson** strategy. The slight decrease to **0.88** in the "Real Scale" is mathematically expected due to the non-linear inverse transformation of residuals, but it represents the honest accuracy for the business (MAE ~13g/km). Technically, the model successfully linearized a complex physical phenomenon.

### 5. Evaluation:

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_lift_and_gains(y_true, y_proba):
    """
    Gera a Tabela de Lift e os Gráficos de Gains/Lift.
    
    Args:
        y_true: Array com os valores reais (0 ou 1)
        y_proba: Array com as probabilidades preditas (predict_proba)
    """
    # 1. Criar DataFrame auxiliar
    df = pd.DataFrame({'y_true': y_true, 'y_proba': y_proba})
    
    # 2. Ordenar por probabilidade (Ranking)
    df = df.sort_values('y_proba', ascending=False)
    
    # 3. Criar os Decis (qcut divide em grupos de tamanho igual)
    df['decile'] = pd.qcut(df['y_proba'].rank(method='first'), 10, labels=False)
    df['decile'] = 10 - df['decile'] # Inverter para que 1 seja o Top Risk
    
    # 4. Agregação (A Mágica acontece aqui)
    lift_table = df.groupby('decile')['y_true'].agg(['count', 'sum', 'mean']).reset_index()
    lift_table.columns = ['Decile', 'Total_Customers', 'Real_Churners', 'Churn_Rate']
    
    # 5. Cálculos de Engenharia
    global_churn_rate = df['y_true'].mean()
    lift_table['Lift'] = lift_table['Churn_Rate'] / global_churn_rate
    
    # Cumulative Gains (Quanto do churn total eu peguei?)
    lift_table['Cumulative_Churners'] = lift_table['Real_Churners'].cumsum()
    lift_table['Total_Churners_In_Base'] = df['y_true'].sum()
    lift_table['Gain_Percentage'] = lift_table['Cumulative_Churners'] / lift_table['Total_Churners_In_Base']
    
    # --- VISUALIZAÇÃO ---
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    # Gráfico de Barras (Lift por Decil)
    sns.barplot(x='Decile', y='Lift', data=lift_table, color='skyblue', alpha=0.7, ax=ax1)
    ax1.axhline(1, color='red', linestyle='--', label='Baseline (Aleatório)')
    ax1.set_ylabel('Lift (x vezes melhor que aleatório)')
    ax1.set_title('Lift Analysis & Cumulative Gains', fontsize=14, fontweight='bold')
    
    # Gráfico de Linha (Ganho Acumulado) - Eixo Secundário
    ax2 = ax1.twinx()
    sns.lineplot(x=lift_table.index, y=lift_table['Gain_Percentage'], color='green', marker='o', ax=ax2, label='% Churn Capturado')
    ax2.set_ylabel('% Total de Churners Capturados')
    ax2.set_ylim(0, 1.1)
    
    # Destaque do KPI "Lift in Top Decile"
    top_decile_gain = lift_table.loc[0, 'Gain_Percentage']
    plt.text(0, top_decile_gain, f'{top_decile_gain:.0%} Capturado', 
             bbox=dict(facecolor='yellow', alpha=0.5))
    
    plt.show()
    
    return lift_table

# Exemplo de chamada no seu notebook:
# y_proba = model.predict_proba(X_test)[:, 1] # Pegar prob da classe 1
# lift_df = plot_lift_and_gains(y_test, y_proba)
# display(lift_df)

In [0]:
# Flatten arrays to ensure 1D dimension.
y_pred_real = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_test_real = y_scaler.inverse_transform(y_test_preprocessed).flatten()

# Waste calculation
residuals = y_test_real - y_pred_real

# --- GRÁFICO A: Residuals vs Predicted ---

# Subplots
fig, ax = plt.subplots(1, 2, figsize = (21, 7))

sns.scatterplot(
    x = y_pred_real,
    y = residuals,
    ax = ax[0],
    alpha = 0.6,
    color = 'steelblue',
    edgecolor = 'black',
    s = 70
)
# Reference Line (Zero Error)
ax[0].axhline(y = 0, color = 'crimson', linestyle = '--', linewidth = 2, label = 'Zero Error' )

std_resid = np.std(residuals)
ax[0].axhline(y = std_resid * 2, color = 'gray', linestyle = ':', alpha = 0.5, label = '+/- Std Dev')
ax[0].axhline(y = -std_resid * 2, color = 'gray', linestyle = ':', alpha = 0.5 )

ax[0].set_title('A. Homoscedasticity: Residuals vs. Predicted Values', fontsize = 14, fontweight = 'bold')
ax[0].set_xlabel('Predicted Emission (g/km)', fontsize = 12)
ax[0].set_ylabel('Error (Real - Predicted)', fontsize = 12)
ax[0].legend()
ax[0].grid(True, alpha = 0.3)


# --- GRAPH B: Distribution of Residuals (The Normality Test)
sns.histplot(
    residuals, 
    kde = True, 
    ax = ax[1],
    color = 'darkslategray',
    edgecolor = 'black',
   
)

mean_resid = np.mean(residuals)
ax[1].axvline(mean_resid, color = 'gold', linestyle = '-', linewidth = 3, label = f'Mean Error: {mean_resid:.2f}')

ax[1].set_title('B. Normality: Error of Distribution', fontsize = 14, fontweight = 'bold')
ax[1].set_xlabel('Magnitude of Error (g/km)', fontsize = 12)
ax[1].set_ylabel('Frequency', fontsize = 12)
ax[1].legend()
ax[1].grid(True, alpha = 0.3)

plt.tight_layout()
plt.show()

#### Key Observations:
---

#### 1. Technical Performance

  - **Explanatory Power (Real R² Score): 0.8844**

  - The model explains **88.4% of the variability** in CO2 emissions using the "Real World" scale (g/km). In the transformed mathematical space (Yeo-Johnson), the fit is even higher (**0.91**), confirming that the non-linear approach successfully captured the physical behavior of the data. This is a significant improvement over the simple univariate model (~0.80).

  - **Margin of Error (MAE): 13.03**

  - The **Mean Absolute Error** indicates that, on average, our predictions deviate by only **13.03 g/km** from the actual value. For a business context where emissions range up to 450 g/km, this represents a very high precision level (approx. 5% relative error), allowing for reliable carbon footprint estimation.

  - **Sensitivity to Large Errors (RMSE vs MAE):**

  * The **RMSE (20.87)** is controlled relative to the MAE (13.03). The gap of ~7.8 points is healthy. It indicates that while there are outliers (likely high-performance sports cars or heavy vehicles), the **Yeo-Johnson transformation** successfully mitigated the extreme penalties that usually distort linear models.
---

#### 2. Model Interpretation

  - The model utilizes a **Power Law** approach (Yeo-Johnson) rather than a simple straight line:

  - **Feature Dominance (Standardized Coefficients):**
  - **Fuel Consumption (Coefficient ~0.71):** This is the dominant driver. The stability analysis showed a variation of only **1.92%** (CV) for this weight, proving it is the most reliable predictor.
  - **Engine Size (Coefficient ~0.27):** This acts as a secondary adjustment factor. Even with a 0.82 correlation to fuel, the model successfully isolated its unique contribution with high stability (CV ~4.05%).

  - **The "Curved" Surface:**
  - Unlike a rigid linear equation, the model projects a **curved surface**. This means it understands that "efficiency" changes as engines get bigger. Physically, this represents the diminishing returns of combustion efficiency in larger engines, providing a much more realistic simulation than a simple linear slope.
---

#### 3. **Conclusion:**

- The Multiple Regression Model with Power Transformation represents the "State-of-the-Art" for this dataset.

- **Strengths:** - **Robustness:** The coefficient stability (CV < 5%) proves the model is immune to multicollinearity. 
- **Physical Coherence:** The residuals follow a near-perfect Gaussian distribution (Mean Bias ~0.81g), indicating that all deterministic signal has been captured.

- **Limitations:**

- **Interpretability:** Because the model operates in a transformed space, we cannot say "1 liter adds X grams" directly. We must use the inverse transformation to get real values.
- **Scope:** The slight residual spread at the high end (>350g/km) suggests that for extreme heavy-duty vehicles, a separate model or additional features (like vehicle weight) might be required.

### 6. Deployment:
---

In [0]:
# DEPLOY PACKAGE: We save everything in a dictionary to ensure integrity.
production_bundle = {
    'model': model, 
    'pt_X': X_scaler, 
    'pt_y': y_scaler,
}

# Saved in a single "pickle" file
joblib.dump(production_bundle, './artifacts/co2_pipeline_v2.pkl')

# Return
print('✅ Complete pipeline saved successfully!')

In [0]:
def predict_emission(engine_size, fuel_consumption):
    
    """
    Performs full inference with Yeo-Johnson pre- and post-processing.
    """
    # 1. Loading (Load the Complete Package)
    bundle = joblib.load('./artifacts/co2_pipeline_v2.pkl')
    model_loaded = bundle['model']
    pt_X_loaded = bundle['pt_X']
    pt_y_loaded = bundle['pt_y']

    # 2. Input Data Engineering
    input_data = pd.DataFrame(
        [[engine_size, fuel_consumption]],
        columns = ['ENGINESIZE', 'FUELCONSUMPTION_COMB']
    )

    # 3. Pre-processing
    input_transformed = pt_X_loaded.transform(input_data)

    # 4. Prediction
    prediction_transformed = model_loaded.predict(input_transformed)

    # 5. Post-processing (Reducing to Grams of CO2)
    prediction_real  = pt_y_loaded.inverse_transform(prediction_transformed.reshape(-1, 1))

    result_g_km = prediction_real[0][0]

    return result_g_km

# --- FINAL TEST (User Acceptance Test) ---

# Scenario: 2.0L car getting 8.5 L/100km
engine = 2.0
consumption = 8.5

try:
    prediction = predict_emission(engine, consumption)

    print(f'--- INFERENCE REPORT ---')
    print(f'Engine: {engine} L')
    print(f'Consumption: {consumption} L/100km')
    print(f'Predicted Emission: {prediction:.2f} g/km')

except Exception as e:
    print(f'Deployment error: {e}')